In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:1


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/atanh'

In [4]:
# !pip install smt

In [5]:
def true_2D_4(xt): #True function for 2D_4 Heat Transfer in a rod x \in [0,1] t \in [0,0.1]
    term1 = 4*u0/np.pi
    
    resol_n = 10000
    
    x = xt[:,0].reshape(-1,1)
    t = xt[:,1].reshape(-1,1)

    u = np.zeros((np.shape(xt)[0],1))
    
    for i in range(resol_n):
        j = 2*i-1
        term2 = np.sin(j*np.pi*x)/j
        term3 = np.exp(-1*np.square(j*np.pi)*t)
        
        u = u + term2*term3
        
    u = term1*u
    
    return u

In [6]:
u0 = 50.0
label = "inv_HT_rowdy"
loss_thresh = 0.1

x_ll = np.array(0.0)
x_ul = np.array(1.0)

x = np.linspace(x_ll,x_ul,100).reshape(-1,1)
t = np.linspace(0,0.1,100).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

u_true = true_2D_4(xt)
u_true_norm = np.linalg.norm(u_true,2)


lb_xt = xt[0]
ub_xt = xt[-1]

xt_test_tensor = torch.from_numpy(xt).float().to(device)

In [7]:
def trainingdata(N_f,N_train,seed):
    
    np.random.seed(seed)
    
    #X_Train
    np.random.seed(seed)
    x_train = np.random.uniform(x_ll,x_ul,(N_train,1))
    t_train = np.random.uniform(0,0.1,(N_train,1))
    
    xt_train = np.hstack((x_train,t_train))
    u_train = true_2D_4(xt_train)
    

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)

    samples = sampling(N_f)
    
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll)) # append training points to collocation points 

    return xt_coll, xt_train, u_train

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val,rowdy_terms):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
        
        self.omega1 = Parameter(torch.ones((len(layers)-2,1))) 
        self.omega1.requiresGrad = True
        
        
        self.alpha = Parameter(torch.zeros(rowdy_terms,len(layers)-2))
        self.alpha.requiresGrad = True
        
        
        self.omega = Parameter((1/n_val)*torch.ones(rowdy_terms,len(layers)-2))
        self.omega.requiresGrad = True
        
        self.n = torch.tensor(n_val)
        
    
        self.lambda1 = Parameter(torch.tensor(0.0))
        self.lambda1.requiresGrad = True
        

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.omega1[i,0]*z)
            for j in range(rowdy_terms):
                a = a + self.alpha[j,i]*self.n*torch.sin((j+1)*self.n*self.omega[j,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    
    def loss_PDE(self, xt_coll,f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_t = autograd.grad(u,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_tt = autograd.grad(u_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dx = u_x_t[:,[0]]
        
        d2u_dx2 = u_xx_tt[:,[0]]
                
        du_dt = u_x_t[:,[1]]
        
        f = du_dt - self.lambda1*d2u_dx2
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_coll,f_hat, xt_train, u_train):

        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_train = self.loss_function(self.forward(xt_train),u_train)
        
        loss_val = loss_f + loss_train
        
        #print(self.iter,"train_loss",loss_train.cpu().detach().numpy(),"F Loss",(loss_f).cpu().detach().numpy())
        return loss_val
     
    'callable for optimizer'                                    
    
    def test(self):
        u_pred = self.forward(xt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
               
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [9]:
def train_step(xt_coll,f_hat, xt_train, u_train,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_coll,f_hat, xt_train, u_train)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [10]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    omega_val.append(PINN.omega.cpu().detach().numpy())
    lambda1_val.append(PINN.lambda1.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [11]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xt_coll, xt_train, u_train = trainingdata(N_f,N_train,123)
    
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_train = torch.from_numpy(xt_train).float().to(device)
    u_train = torch.from_numpy(u_train).float().to(device)
    
    
    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    
    for i in range(max_iter):
        train_step(xt_coll,f_hat, xt_train, u_train,i)

        loss_np = PINN.loss(xt_coll,f_hat, xt_train, u_train).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1],"Lambda1",lambda1_val[-1])

    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))
    

In [12]:
max_reps = 10#10
max_iter = 200 #75

train_loss_full = []
test_mse_full = []
test_re_full = []
alpha_full = []
omega_full = []
lambda1_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

n_val =1.0
rowdy_terms = 2

for reps in range(max_reps):
    print(label)
    'Generate Training data'
    print(reps)
    torch.manual_seed(reps*36)

    train_loss = []
    test_mse_loss = []
    test_re_loss = []   
    alpha_val = []
    omega_val = []
    lambda1_val = []

    N_f = 50000 #Total number of collocation points 
    N_train = 5000

    layers = np.array([2,50,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

    PINN = Sequentialmodel(layers,n_val,rowdy_terms)

    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())

    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.25, 
                              max_iter = 20, 
                              max_eval = 30, 
                              tolerance_grad = 1e-8, 
                              tolerance_change = 1e-8, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')


    nan_flag = train_model(max_iter,reps)

    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time
    alpha_full.append(alpha_val)
    omega_full.append(omega_val)
    lambda1_full.append(lambda1_val)


    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "alpha": alpha_full,"omega": omega_full,"lambda1": lambda1_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic) 


inv_HT_rowdy
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 856.28436 Test MSE 856.281854632382 Test RE 0.49260355436263215 Lambda1 -0.060201727
1 Train Loss 832.81964 Test MSE 824.5255512458574 Test RE 0.48338284104154816 Lambda1 -0.0669174
2 Train Loss 757.71063 Test M

77 Train Loss 57.55304 Test MSE 57.29496307762919 Test RE 0.12742287291070326 Lambda1 0.47501564
78 Train Loss 56.931114 Test MSE 56.14829050440956 Test RE 0.126141339870504 Lambda1 0.48195142
79 Train Loss 56.253033 Test MSE 55.604251842785125 Test RE 0.12552874054128146 Lambda1 0.48496822
80 Train Loss 55.68016 Test MSE 54.749396267721046 Test RE 0.12456006835147833 Lambda1 0.48980248
81 Train Loss 55.19895 Test MSE 54.206765805330235 Test RE 0.12394126338383632 Lambda1 0.49413314
82 Train Loss 54.740166 Test MSE 53.71027769821896 Test RE 0.12337235916779964 Lambda1 0.4962427
83 Train Loss 54.085953 Test MSE 52.917307103002464 Test RE 0.12245824703692625 Lambda1 0.50102186
84 Train Loss 53.573174 Test MSE 52.69928065792586 Test RE 0.12220571440113899 Lambda1 0.50312334
85 Train Loss 53.164204 Test MSE 52.33578420230864 Test RE 0.12178352449144766 Lambda1 0.505277
86 Train Loss 52.77444 Test MSE 51.85211477926768 Test RE 0.12121947743738462 Lambda1 0.5091128
87 Train Loss 52.6487 Test

161 Train Loss 35.49566 Test MSE 36.470583369032035 Test RE 0.10166243923345093 Lambda1 0.6742605
162 Train Loss 35.221207 Test MSE 36.279029431277266 Test RE 0.10139510763803541 Lambda1 0.6764082
163 Train Loss 34.924862 Test MSE 36.08261330559487 Test RE 0.10112025652228515 Lambda1 0.67911094
164 Train Loss 34.679047 Test MSE 36.003711869479375 Test RE 0.10100963676465367 Lambda1 0.6795372
165 Train Loss 34.4626 Test MSE 35.763241143080705 Test RE 0.10067174722774691 Lambda1 0.68287283
166 Train Loss 34.292892 Test MSE 35.59848030412347 Test RE 0.10043958273581352 Lambda1 0.68521297
167 Train Loss 34.243416 Test MSE 35.55073421573947 Test RE 0.1003722033667512 Lambda1 0.68642133
168 Train Loss 34.21713 Test MSE 35.52000516394549 Test RE 0.10032881453812516 Lambda1 0.6872072
169 Train Loss 34.159313 Test MSE 35.61470534343577 Test RE 0.10046246925397807 Lambda1 0.6859534
170 Train Loss 34.11338 Test MSE 35.459789086764864 Test RE 0.10024373614016306 Lambda1 0.6891276
171 Train Loss 34

37 Train Loss 34.063187 Test MSE 37.82641172880244 Test RE 0.10353489403240985 Lambda1 0.807009
38 Train Loss 33.839424 Test MSE 37.34423580533554 Test RE 0.10287289434908282 Lambda1 0.82499653
39 Train Loss 33.480133 Test MSE 36.81867891149096 Test RE 0.10214644850876659 Lambda1 0.84021056
40 Train Loss 33.321663 Test MSE 36.798871031134375 Test RE 0.10211896820677557 Lambda1 0.84094095
41 Train Loss 33.024643 Test MSE 36.75842280082404 Test RE 0.10206282970180172 Lambda1 0.84388375
42 Train Loss 32.74638 Test MSE 36.61766262040882 Test RE 0.10186722606823014 Lambda1 0.8537569
43 Train Loss 32.68953 Test MSE 36.50788691596815 Test RE 0.10171441811578287 Lambda1 0.857277
44 Train Loss 32.633312 Test MSE 36.46627892621426 Test RE 0.10165643969863854 Lambda1 0.85774934
45 Train Loss 32.59558 Test MSE 36.416466505790034 Test RE 0.10158698534384614 Lambda1 0.86039925
46 Train Loss 32.50282 Test MSE 36.17969986569927 Test RE 0.10125620597224018 Lambda1 0.86663634
47 Train Loss 32.435375 Tes

122 Train Loss 29.057074 Test MSE 32.66841149379512 Test RE 0.09621730299526626 Lambda1 0.957203
123 Train Loss 29.057074 Test MSE 32.66841149379512 Test RE 0.09621730299526626 Lambda1 0.957203
124 Train Loss 29.057074 Test MSE 32.66841149379512 Test RE 0.09621730299526626 Lambda1 0.957203
125 Train Loss 29.057074 Test MSE 32.66841149379512 Test RE 0.09621730299526626 Lambda1 0.957203
126 Train Loss 29.057074 Test MSE 32.66841149379512 Test RE 0.09621730299526626 Lambda1 0.957203
127 Train Loss 29.057074 Test MSE 32.66841149379512 Test RE 0.09621730299526626 Lambda1 0.957203
128 Train Loss 29.057074 Test MSE 32.66841149379512 Test RE 0.09621730299526626 Lambda1 0.957203
129 Train Loss 29.057074 Test MSE 32.66841149379512 Test RE 0.09621730299526626 Lambda1 0.957203
130 Train Loss 29.057074 Test MSE 32.66841149379512 Test RE 0.09621730299526626 Lambda1 0.957203
131 Train Loss 29.057074 Test MSE 32.66841149379512 Test RE 0.09621730299526626 Lambda1 0.957203
132 Train Loss 29.057074 Test 

...

13 Train Loss 238.95068 Test MSE 239.0889016553787 Test RE 0.26029682998109743 Lambda1 0.11098295
14 Train Loss 229.45401 Test MSE 231.75484317286717 Test RE 0.2562734290556248 Lambda1 0.14089748
15 Train Loss 222.28923 Test MSE 221.35633690109452 Test RE 0.25045813957673524 Lambda1 0.18779756
16 Train Loss 217.49713 Test MSE 215.69208825298432 Test RE 0.24723290860504618 Lambda1 0.2115823
17 Train Loss 210.02245 Test MSE 207.40498633742286 Test RE 0.2424369252068467 Lambda1 0.23143113
18 Train Loss 198.72029 Test MSE 195.3855626259385 Test RE 0.2353073026645768 Lambda1 0.2741727
19 Train Loss 182.80391 Test MSE 175.8223784709418 Test RE 0.22321647524194194 Lambda1 0.28241214
20 Train Loss 161.63368 Test MSE 156.357220055084 Test RE 0.21049808091883737 Lambda1 0.2968876
21 Train Loss 146.10225 Test MSE 135.4825392502006 Test RE 0.1959434879136258 Lambda1 0.30554304
22 Train Loss 128.78685 Test MSE 112.88698993034454 Test RE 0.17885913130746164 Lambda1 0.32737055
23 Train Loss 113.56748

98 Train Loss 27.504045 Test MSE 30.471902142582515 Test RE 0.09292636583991054 Lambda1 0.91452116
99 Train Loss 27.467125 Test MSE 30.45844491980251 Test RE 0.09290584416556587 Lambda1 0.9145075
100 Train Loss 27.443773 Test MSE 30.460130906799076 Test RE 0.09290841547014196 Lambda1 0.9155562
101 Train Loss 27.387518 Test MSE 30.337072839174827 Test RE 0.09272055185140092 Lambda1 0.9204262
102 Train Loss 27.26428 Test MSE 30.118459791334555 Test RE 0.09238586940421431 Lambda1 0.92668045
103 Train Loss 27.144928 Test MSE 29.998899387730223 Test RE 0.0922023162666231 Lambda1 0.93466353
104 Train Loss 27.105528 Test MSE 29.92403927088804 Test RE 0.09208720225035014 Lambda1 0.93911535
105 Train Loss 27.083447 Test MSE 29.86615087595542 Test RE 0.09199808725998458 Lambda1 0.9422255
106 Train Loss 27.055025 Test MSE 29.84190401229373 Test RE 0.09196073530904234 Lambda1 0.9427863
107 Train Loss 26.982363 Test MSE 29.69833485104918 Test RE 0.09173925742519855 Lambda1 0.94917995
108 Train Loss

181 Train Loss 25.820301 Test MSE 28.990962000572633 Test RE 0.09064012253609548 Lambda1 0.98328465
182 Train Loss 25.820301 Test MSE 28.990962000572633 Test RE 0.09064012253609548 Lambda1 0.98328465
183 Train Loss 25.820301 Test MSE 28.990962000572633 Test RE 0.09064012253609548 Lambda1 0.98328465
184 Train Loss 25.820301 Test MSE 28.990962000572633 Test RE 0.09064012253609548 Lambda1 0.98328465
185 Train Loss 25.820301 Test MSE 28.990962000572633 Test RE 0.09064012253609548 Lambda1 0.98328465
186 Train Loss 25.820301 Test MSE 28.990962000572633 Test RE 0.09064012253609548 Lambda1 0.98328465
187 Train Loss 25.820301 Test MSE 28.990962000572633 Test RE 0.09064012253609548 Lambda1 0.98328465
188 Train Loss 25.820301 Test MSE 28.990962000572633 Test RE 0.09064012253609548 Lambda1 0.98328465
189 Train Loss 25.820301 Test MSE 28.990962000572633 Test RE 0.09064012253609548 Lambda1 0.98328465
190 Train Loss 25.820301 Test MSE 28.990962000572633 Test RE 0.09064012253609548 Lambda1 0.98328465


57 Train Loss 30.567392 Test MSE 34.03600777901071 Test RE 0.09821062565744641 Lambda1 0.91561806
58 Train Loss 30.534641 Test MSE 34.00462839956647 Test RE 0.09816534274491079 Lambda1 0.9168202
59 Train Loss 30.51318 Test MSE 34.007343114732954 Test RE 0.09816926111777334 Lambda1 0.91639054
60 Train Loss 30.436317 Test MSE 33.89356197161853 Test RE 0.09800489706689358 Lambda1 0.92465854
61 Train Loss 30.376461 Test MSE 33.87687706539667 Test RE 0.09798077148502755 Lambda1 0.91962487
62 Train Loss 30.3529 Test MSE 33.89285033416371 Test RE 0.09800386819418817 Lambda1 0.9169118
63 Train Loss 30.233076 Test MSE 33.75644269696972 Test RE 0.097806452603048 Lambda1 0.9314587
64 Train Loss 30.158358 Test MSE 33.59927350897798 Test RE 0.09757849470248305 Lambda1 0.9423337
65 Train Loss 30.102716 Test MSE 33.602156966198244 Test RE 0.09758268165876785 Lambda1 0.94585514
66 Train Loss 30.065825 Test MSE 33.52952361045885 Test RE 0.09747715879082793 Lambda1 0.9527236
67 Train Loss 30.027185 Test

141 Train Loss 29.987356 Test MSE 33.48518964420066 Test RE 0.09741269353638696 Lambda1 0.9541989
142 Train Loss 29.987356 Test MSE 33.48518964420066 Test RE 0.09741269353638696 Lambda1 0.9541989
143 Train Loss 29.987356 Test MSE 33.48518964420066 Test RE 0.09741269353638696 Lambda1 0.9541989
144 Train Loss 29.987356 Test MSE 33.48518964420066 Test RE 0.09741269353638696 Lambda1 0.9541989
145 Train Loss 29.987356 Test MSE 33.48518964420066 Test RE 0.09741269353638696 Lambda1 0.9541989
146 Train Loss 29.987356 Test MSE 33.48518964420066 Test RE 0.09741269353638696 Lambda1 0.9541989
147 Train Loss 29.987356 Test MSE 33.48518964420066 Test RE 0.09741269353638696 Lambda1 0.9541989
148 Train Loss 29.987356 Test MSE 33.48518964420066 Test RE 0.09741269353638696 Lambda1 0.9541989
149 Train Loss 29.987356 Test MSE 33.48518964420066 Test RE 0.09741269353638696 Lambda1 0.9541989
150 Train Loss 29.987356 Test MSE 33.48518964420066 Test RE 0.09741269353638696 Lambda1 0.9541989
151 Train Loss 29.98

In [13]:
import scipy.io as sio
import numpy as np

In [14]:
for tune_reps in range(25):
    label = "inv_HT_rowdy_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

FileNotFoundError: [Errno 2] No such file or directory: 'inv_HT_rowdy_tune0.mat'

In [ ]:
lrnr_tune[10]